## 1. Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm
import pandas as pd
import random
import shutil
from datetime import datetime

# Configure matplotlib for inline display in notebook
%matplotlib inline

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 10)


print('✓ Libraries imported successfully')

## 2. Configuration

In [ ]:
# Base directories
BASE_DIR = Path.cwd().parent
MODEL_NAME = "yolov8l"  # Model name without .pt extension

# Choose YOLO model versions that are fully supported with ultralytics:
# ✅ YOLOv8: 'yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x'
# ✅ YOLOv9: 'yolov9s', 'yolov9m', 'yolov9l', 'yolov9x'
# ✅ YOLOv10: 'yolov10n', 'yolov10s', 'yolov10m', 'yolov10l', 'yolov10x'
# ✅ YOLO11: 'yolo11n', 'yolo11s', 'yolo11m', 'yolo11l', 'yolo11x'
# ✅ YOLO12: 'yolo12n', 'yolo12s', 'yolo12m', 'yolo12l', 'yolo12x'
#
# Model sizes: n=nano, s=small, m=medium, l=large, x=extra-large

MODELS_DIR = BASE_DIR / 'models' / MODEL_NAME
TMP_DIR = BASE_DIR / 'tmp' / MODEL_NAME
RUNS_DIR = BASE_DIR / 'quick_test' / 'runs'

# Dataset Selection - Choose one:
# Option 1: Full dataset (~100k images)
# YOLO_DATASET_ROOT = BASE_DIR / 'bdd100k_yolo'
# DATA_YAML_PATH = YOLO_DATASET_ROOT / 'data.yaml'

# Option 2: Limited dataset (representative samples - for quick testing)
YOLO_DATASET_ROOT = BASE_DIR / 'bdd100k_yolo_limited'
DATA_YAML_PATH = YOLO_DATASET_ROOT / 'data.yaml'

# Choose data split
USED_DATA_SPLIT = "test"  # 'train', 'val', or 'test'

# Dataset paths
IMAGES_DIR = YOLO_DATASET_ROOT / 'images' / USED_DATA_SPLIT
LABELS_DIR = YOLO_DATASET_ROOT / 'labels' / USED_DATA_SPLIT

# Verify dataset exists
if not DATA_YAML_PATH.exists():
    raise FileNotFoundError(
        f"Dataset not found: {DATA_YAML_PATH}\n\n"
        f"Please run the dataset preparation script first:\n"
        f"  python3 process_bdd100k_to_yolo_dataset.py\n"
    )

# Generate timestamp and run name
RUN_TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
RUN_NAME = f'{MODEL_NAME}_{YOLO_DATASET_ROOT.name}_{USED_DATA_SPLIT}'

# Create run-specific directory
RUN_DIR = RUNS_DIR / RUN_NAME
RUN_DIR.mkdir(parents=True, exist_ok=True)
TMP_DIR.mkdir(parents=True, exist_ok=True)
RUNS_DIR.mkdir(parents=True, exist_ok=True)

# Load class names from data.yaml
import yaml
with open(DATA_YAML_PATH, 'r') as f:
    dataset_config = yaml.safe_load(f)
    CLASS_NAMES = {i: name for i, name in enumerate(dataset_config['names'])}

NUM_CLASSES = len(CLASS_NAMES)

# Generate colors for each class
np.random.seed(42)
COLORS = {idx: tuple(np.random.randint(50, 255, 3).tolist()) for idx in CLASS_NAMES.keys()}

print('✓ Configuration loaded')
print(f'  Model: {MODEL_NAME}')
print(f'  Dataset: {YOLO_DATASET_ROOT.name}')
print(f'  Split: {USED_DATA_SPLIT}')
print(f'  Images: {IMAGES_DIR}')
print(f'  Labels: {LABELS_DIR}')
print(f'  Classes: {NUM_CLASSES}')
print(f'  Run directory: {RUN_DIR}')

## 3. Load YOLO Model

In [ ]:
# Load YOLO model with automatic download
model_path = MODELS_DIR / f'{MODEL_NAME}.pt'

if not model_path.exists():
    print(f'Model not found at {model_path}')
    print(f'Downloading {MODEL_NAME} ...')
    
    try:
        # Download model - it will be cached by ultralytics
        MODEL_NAME_n = MODEL_NAME 
        if MODEL_NAME.startswith('yolov11') or MODEL_NAME.startswith('yolov12'):
            MODEL_NAME_n = MODEL_NAME + '.pt'
        model = YOLO(MODEL_NAME_n)
        
        # Create models directory
        MODELS_DIR.mkdir(parents=True, exist_ok=True)
        
        # Save model to our directory using export/save
        try:
            # Try to save using the model's save method
            if hasattr(model, 'save'):
                model.save(str(model_path))
                print(f'✓ Model downloaded and saved to {model_path}')
                print(f'  Size: {model_path.stat().st_size / (1024*1024):.1f} MB')
            else:
                # Fallback: copy from cache
                import glob
                cache_patterns = [
                    str(Path.home() / '.cache' / 'ultralytics' / '**' / f'{MODEL_NAME}.pt'),
                    str(Path.home() / '.config' / 'Ultralytics' / '**' / f'{MODEL_NAME}.pt'),
                ]
                
                model_found = False
                for pattern in cache_patterns:
                    cache_paths = glob.glob(pattern, recursive=True)
                    if cache_paths:
                        shutil.copy(cache_paths[0], model_path)
                        print(f'✓ Model downloaded and saved to {model_path}')
                        print(f'  Size: {model_path.stat().st_size / (1024*1024):.1f} MB')
                        model_found = True
                        break
                
                if not model_found:
                    print(f'✓ Model loaded from ultralytics cache')
                    print(f'  Note: Model is in cache, not copied to {model_path}')
                    print(f'  This is normal and the model will work correctly')
        except Exception as save_error:
            print(f'⚠️  Could not save model to custom location: {save_error}')
            print(f'✓ Model loaded successfully from ultralytics cache')
            
    except Exception as e:
        print(f'\n❌ Error downloading model: {e}')
        raise
else:
    model = YOLO(str(model_path))
    print(f'✓ Model loaded from {model_path}')

# Get model information
model_params = sum(p.numel() for p in model.model.parameters())
model_size_mb = model_path.stat().st_size / (1024*1024) if model_path.exists() else 0

# Calculate FLOPs using model.info() method which is more accurate
try:
    from ultralytics.utils.torch_utils import get_flops
    # FLOPs for 640x640 input (standard YOLO input size)
    flops = get_flops(model.model, imgsz=(1, 3, 640, 640))
    flops_gflops = flops / 1e9
except Exception as e:
    print(f'⚠️  Could not calculate FLOPs: {e}')
    flops_gflops = 0

print(f'\n📊 Model Information:')
print(f'  Model: {MODEL_NAME}')
print(f'  Classes in model: {len(model.names)}')
print(f'  Task: {model.task}')
print(f'  Parameters: {model_params / 1e6:.1f}M')
print(f'  Model Size: {model_size_mb:.1f} MB')
print(f'  FLOPs (640x640): {flops_gflops:.2f} GFLOPs')


## 4. Load Dataset

In [ ]:
import json

# Get all images and labels
image_files = sorted(list(IMAGES_DIR.glob('*.jpg')) + list(IMAGES_DIR.glob('*.png')))
label_files = sorted([LABELS_DIR / f'{img.stem}.txt' for img in image_files if (LABELS_DIR / f'{img.stem}.txt').exists()])

# Filter to only images with labels
valid_images = [img for img in image_files if (LABELS_DIR / f'{img.stem}.txt').exists()]

print(f'✓ Dataset loaded')
print(f'  Total images: {len(image_files)}')
print(f'  Images with labels: {len(valid_images)}')
print(f'  Label files: {len(label_files)}')

# Load performance metadata for per-image attribute analysis
METADATA_DIR = YOLO_DATASET_ROOT / 'representative_json'
PERFORMANCE_FILE = METADATA_DIR / f'{USED_DATA_SPLIT}_performance_analysis.json'

if PERFORMANCE_FILE.exists():
    with open(PERFORMANCE_FILE, 'r') as f:
        performance_data = json.load(f)
    
    print(f'\n✓ Performance metadata loaded: {PERFORMANCE_FILE.name}')
    print(f'  Images with attributes: {performance_data["total_images"]}')
    print(f'  Attributes available: weather, scene, timeofday')
    print(f'  Per-image class distribution available')
    
    # Create quick lookup dictionary for attributes
    image_attributes = {img['basename']: img for img in performance_data['images']}
else:
    print(f'\n⚠️  Performance metadata not found: {PERFORMANCE_FILE}')
    print(f'  Attribute-based analysis will not be available')
    print(f'  Run: python3 process_bdd100k_to_yolo_dataset.py')
    performance_data = None
    image_attributes = {}

## 6. Run Official YOLO Validation

### IMPORTANT: Optimized Validation Approach
This notebook uses **YOLO's official validation method** to calculate all metrics, confusion matrix, and predictions in a single pass. This approach:
- ✅ **Faster**: Single validation pass instead of multiple loops
- ✅ **No Duplicates**: Uses YOLO's built-in validation logic
- ✅ **Official Metrics**: Provides mAP, precision, recall directly from YOLO
- ✅ **Confusion Matrix**: Extracted from YOLO validation results

In [ ]:
# Run official YOLO validation with W&B tracking
import time
print('=' * 80)
print('RUNNING OFFICIAL YOLO VALIDATION WITH W&B TRACKING')
print('=' * 80)

# Create a dataset structure that YOLO expects
validation_dataset_root = TMP_DIR / 'yolo_validation_dataset'
validation_images_dir = validation_dataset_root / 'images' / USED_DATA_SPLIT
validation_labels_dir = validation_dataset_root / 'labels' / USED_DATA_SPLIT

# Create directories
validation_images_dir.mkdir(parents=True, exist_ok=True)
validation_labels_dir.mkdir(parents=True, exist_ok=True)

# Create symbolic links to images and labels
import shutil
print(f'Setting up validation dataset structure...')

# Link images
for img_file in IMAGES_DIR.glob('*.jpg'):
    link_path = validation_images_dir / img_file.name
    if not link_path.exists():
        try:
            link_path.symlink_to(img_file)
        except:
            shutil.copy2(img_file, link_path)

for img_file in IMAGES_DIR.glob('*.png'):
    link_path = validation_images_dir / img_file.name
    if not link_path.exists():
        try:
            link_path.symlink_to(img_file)
        except:
            shutil.copy2(img_file, link_path)

# Link labels
for label_file in LABELS_DIR.glob('*.txt'):
    if label_file.name != 'classes.txt':
        link_path = validation_labels_dir / label_file.name
        if not link_path.exists():
            try:
                link_path.symlink_to(label_file)
            except:
                shutil.copy2(label_file, link_path)

print(f'✓ Validation dataset prepared')
print(f'  Images: {len(list(validation_images_dir.glob("*")))}')
print(f'  Labels: {len(list(validation_labels_dir.glob("*.txt")))}')

# Create data.yaml file
data_yaml_path = validation_dataset_root / 'data.yaml'
iou_threshold = 0.5
data_yaml_content = f"""path: {validation_dataset_root}
train: images/train
val: images/{USED_DATA_SPLIT}
test: images/test

nc: {NUM_CLASSES}
names: {list(CLASS_NAMES.values())}
"""

with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f'✓ Created data.yaml at: {data_yaml_path}')

# Initialize Weights & Biases
try:
    import wandb
    wandb.init(
        project="yolo-bdd100k-validation",
        name=W_B_RUN_NAME,
        config={
            "model": MODEL_NAME,
            "dataset": USED_DATASET,
            "split": USED_DATA_SPLIT,
            "iou_threshold": iou_threshold,
            "num_classes": NUM_CLASSES,
            "model_params": model_params,
            "model_size_mb": model_size_mb,
            "flops_gflops": flops_gflops
        }
    )
    print(f'\n✓ Weights & Biases initialized: {W_B_RUN_NAME}')
    print(f'  Project: yolo-bdd100k-validation')
    print(f'  Run: {W_B_RUN_NAME}')
except ImportError:
    print('\n⚠️  Weights & Biases not available. Install with: pip install wandb')
    wandb = None
except Exception as e:
    print(f'\n⚠️  W&B initialization error: {e}')
    print('  Continuing without W&B tracking...')
    wandb = None

# Run validation with timing
print('\nRunning YOLO validation...')
start_time = time.time()

validation_results = model.val(
    data=str(data_yaml_path),
    split=USED_DATA_SPLIT,
    save_json=False,
    save_txt=False,
    conf=0.001,
    iou=iou_threshold,
    verbose=True,
    plots=True,
    project=str(RUN_DIR),
    name='yolo_validation'
)

end_time = time.time()
total_time = end_time - start_time

# Extract speed metrics from YOLO validation
num_images = len(list(validation_images_dir.glob("*.jpg"))) + len(list(validation_images_dir.glob("*.png")))
avg_inference_time = total_time / num_images if num_images > 0 else 0
fps = 1 / avg_inference_time if avg_inference_time > 0 else 0

# Extract overall metrics
yolo_metrics = {
    'precision': float(validation_results.box.mp),
    'recall': float(validation_results.box.mr),
    'map50': float(validation_results.box.map50),
    'map50_95': float(validation_results.box.map),
    'fitness': float(validation_results.fitness)
}

# Extract per-class metrics from YOLO
yolo_class_metrics = {}
class_tp = {}
class_fp = {}
class_fn = {}

if hasattr(validation_results.box, 'ap_class_index') and len(validation_results.box.ap_class_index) > 0:
    for i, class_idx in enumerate(validation_results.box.ap_class_index):
        class_idx = int(class_idx)
        class_name = CLASS_NAMES.get(class_idx, f'class_{class_idx}')
        
        precision = float(validation_results.box.p[i]) if i < len(validation_results.box.p) else 0.0
        recall = float(validation_results.box.r[i]) if i < len(validation_results.box.r) else 0.0
        ap50 = float(validation_results.box.ap50[i]) if i < len(validation_results.box.ap50) else 0.0
        ap50_95 = float(validation_results.box.ap[i]) if i < len(validation_results.box.ap) else 0.0
        
        yolo_class_metrics[class_name] = {
            'precision': precision,
            'recall': recall,
            'ap50': ap50,
            'ap50_95': ap50_95
        }
        
        class_tp[class_idx] = 0
        class_fp[class_idx] = 0
        class_fn[class_idx] = 0

# Extract confusion matrix from YOLO validation results
confusion_matrix = validation_results.confusion_matrix.matrix if hasattr(validation_results, 'confusion_matrix') else np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=int)

# Verify confusion matrix exists
if confusion_matrix is not None and confusion_matrix.size > 0:
    print(f'\n✓ Confusion matrix extracted successfully')
    print(f'  Shape: {confusion_matrix.shape}')
    print(f'  Diagonal sum (correct): {np.trace(confusion_matrix)}')
else:
    print(f'\n⚠️  Warning: Confusion matrix is empty')

# Calculate TP, FP, FN from confusion matrix
for i in range(NUM_CLASSES):
    class_tp[i] = int(confusion_matrix[i, i]) if i < confusion_matrix.shape[0] and i < confusion_matrix.shape[1] else 0
    
    if i < confusion_matrix.shape[1]:
        class_fp[i] = int(confusion_matrix[:, i].sum() - confusion_matrix[i, i])
    else:
        class_fp[i] = 0
    
    if i < confusion_matrix.shape[0]:
        class_fn[i] = int(confusion_matrix[i, :].sum() - confusion_matrix[i, i])
    else:
        class_fn[i] = 0

# Create results_data for compatibility with report generation
results_data = []
for img_path in validation_images_dir.glob("*.jpg"):
    results_data.append({'image_path': img_path})
for img_path in validation_images_dir.glob("*.png"):
    results_data.append({'image_path': img_path})

print('\n' + '=' * 80)
print('OFFICIAL YOLO VALIDATION RESULTS')
print('=' * 80)
print(f"Precision (mean): {yolo_metrics['precision']:.4f}")
print(f"Recall (mean):    {yolo_metrics['recall']:.4f}")
print(f"mAP@0.5:          {yolo_metrics['map50']:.4f}")
print(f"mAP@0.5:0.95:     {yolo_metrics['map50_95']:.4f}")
print(f"Fitness:          {yolo_metrics['fitness']:.4f}")
print(f'\n⚡ Performance Metrics:')
print(f'  Total Time: {total_time:.2f}s')
print(f'  Average Inference Time: {avg_inference_time*1000:.2f}ms per image')
print(f'  FPS (Frames Per Second): {fps:.2f}')
print('=' * 80)

# Log comprehensive metrics to Weights & Biases
if wandb:
    try:
        # Log overall metrics
        wandb.log({
            # Accuracy metrics
            "metrics/precision": yolo_metrics['precision'],
            "metrics/recall": yolo_metrics['recall'],
            "metrics/mAP@0.5": yolo_metrics['map50'],
            "metrics/mAP@0.5:0.95": yolo_metrics['map50_95'],
            "metrics/fitness": yolo_metrics['fitness'],
            
            # Performance metrics
            "performance/total_time_seconds": total_time,
            "performance/avg_inference_time_ms": avg_inference_time * 1000,
            "performance/fps": fps,
            "performance/images_processed": num_images,
            
            # Model info
            "model/parameters_millions": model_params / 1e6,
            "model/size_mb": model_size_mb,
            "model/flops_gflops": flops_gflops
        })
        
        # Log per-class metrics
        for class_name, metrics in yolo_class_metrics.items():
            wandb.log({
                f"class/{class_name}/precision": metrics['precision'],
                f"class/{class_name}/recall": metrics['recall'],
                f"class/{class_name}/ap50": metrics['ap50'],
                f"class/{class_name}/ap50_95": metrics['ap50_95']
            })
        
        print(f'\n✓ Metrics logged to Weights & Biases')
        print(f'  Dashboard: https://wandb.ai/yolo-bdd100k-validation/{W_B_RUN_NAME}')
    except Exception as e:
        print(f'\n⚠️  Error logging to W&B: {e}')

# Save YOLO validation results with performance metrics
yolo_results_path = RUN_DIR / 'yolo_validation_metrics.json'
import json
with open(yolo_results_path, 'w') as f:
    json.dump({
        'overall_metrics': yolo_metrics,
        'per_class_metrics': yolo_class_metrics,
        'performance': {
            'total_time_seconds': float(total_time),
            'avg_inference_time_ms': float(avg_inference_time * 1000),
            'fps': float(fps),
            'images_processed': int(num_images)
        },
        'model_info': {
            'parameters': int(model_params),
            'model_size_mb': float(model_size_mb),
            'flops_gflops': float(flops_gflops)
        }
    }, f, indent=2)

print(f'\n✓ YOLO validation results saved to: {yolo_results_path}')
print(f'✓ YOLO validation plots saved to: {RUN_DIR / "yolo_validation"}')
print(f'✓ Confusion matrix extracted and will be visualized in next cell')
print(f'  Diagonal sum (correct predictions): {np.trace(confusion_matrix)}')
print(f'  Total predictions: {confusion_matrix.sum()}')

# Finish W&B run
if wandb:
    try:
        wandb.finish()
        print(f'\n✓ Weights & Biases run completed successfully')
    except Exception as e:
        print(f'\n⚠️  Error finishing W&B run: {e}')


## 7. Display Results Summary

In [ ]:
# Calculate per-class metrics from YOLO validation results
metrics_data = []

for class_id in sorted(CLASS_NAMES.keys()):
    tp = class_tp.get(class_id, 0)
    fp = class_fp.get(class_id, 0)
    fn = class_fn.get(class_id, 0)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    metrics_data.append({
        'Class': CLASS_NAMES[class_id],
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })

df_metrics = pd.DataFrame(metrics_data)

# Overall metrics
total_tp = sum(class_tp.values())
total_fp = sum(class_fp.values())
total_fn = sum(class_fn.values())

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

print('=' * 80)
print('YOLOv8 VALIDATION RESULTS SUMMARY')
print('=' * 80)
print(f'\\nDataset: BDD100K Limited - Validation Split')
print(f'Images processed: {len(list(validation_images_dir.glob("*")))}')
print(f'IoU Threshold: {iou_threshold}')
print(f'\\nOVERALL METRICS (From YOLO Validation):')
print(f'  Precision: {yolo_metrics["precision"]:.4f}')
print(f'  Recall:    {yolo_metrics["recall"]:.4f}')
print(f'  mAP@0.5:   {yolo_metrics["map50"]:.4f}')
print(f'  mAP@0.5:0.95: {yolo_metrics["map50_95"]:.4f}')
print(f'\\nOVERALL METRICS (From Confusion Matrix):')
print(f'  Precision: {overall_precision:.4f}')
print(f'  Recall:    {overall_recall:.4f}')
print(f'  F1-Score:  {overall_f1:.4f}')
print(f'\\nPER-CLASS METRICS:')
print(df_metrics.to_string(index=False))
print('\\n' + '=' * 80)

## 8. Visualize Metrics

In [ ]:
# Plot metrics
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Precision by class
ax1 = axes[0, 0]
df_metrics_sorted = df_metrics.sort_values('Precision', ascending=True)
ax1.barh(df_metrics_sorted['Class'], df_metrics_sorted['Precision'], color='skyblue')
ax1.set_xlabel('Precision', fontweight='bold')
ax1.set_title('Precision by Class', fontweight='bold', fontsize=14)
ax1.set_xlim([0, 1])
ax1.grid(axis='x', alpha=0.3)

# Recall by class
ax2 = axes[0, 1]
df_metrics_sorted = df_metrics.sort_values('Recall', ascending=True)
ax2.barh(df_metrics_sorted['Class'], df_metrics_sorted['Recall'], color='lightcoral')
ax2.set_xlabel('Recall', fontweight='bold')
ax2.set_title('Recall by Class', fontweight='bold', fontsize=14)
ax2.set_xlim([0, 1])
ax2.grid(axis='x', alpha=0.3)

# F1-Score by class
ax3 = axes[1, 0]
df_metrics_sorted = df_metrics.sort_values('F1-Score', ascending=True)
ax3.barh(df_metrics_sorted['Class'], df_metrics_sorted['F1-Score'], color='lightgreen')
ax3.set_xlabel('F1-Score', fontweight='bold')
ax3.set_title('F1-Score by Class', fontweight='bold', fontsize=14)
ax3.set_xlim([0, 1])
ax3.grid(axis='x', alpha=0.3)

# Detection counts
ax4 = axes[1, 1]
counts = [class_tp[i] + class_fn[i] for i in sorted(CLASS_NAMES.keys())]
class_labels = [CLASS_NAMES[i] for i in sorted(CLASS_NAMES.keys())]
ax4.bar(class_labels, counts, color='mediumpurple')
ax4.set_xlabel('Class', fontweight='bold')
ax4.set_ylabel('Total Objects (TP + FN)', fontweight='bold')
ax4.set_title('Ground Truth Object Counts', fontweight='bold', fontsize=14)
ax4.tick_params(axis='x', rotation=45)
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()

# Save for report
metrics_fig_path = RUN_DIR / 'metrics_charts.png'
plt.savefig(metrics_fig_path, dpi=150, bbox_inches='tight')

# Show in notebook
plt.show()


## 8.5. Confusion Matrix

In [ ]:
# Visualize confusion matrix from YOLO validation
from matplotlib.patches import Rectangle

# Create larger figure
fig, ax = plt.subplots(1, 1, figsize=(16, 14))

# Draw each cell manually with solid colors
for i in range(NUM_CLASSES):
    for j in range(NUM_CLASSES):
        value = confusion_matrix[i, j]
        
        # Determine cell color
        if value == 0:
            # White for empty cells
            cell_color = 'white'
        elif i == j:
            # Green for diagonal (True Positives)
            cell_color = '#00CC00'
        else:
            # Red for off-diagonal (False Positives/Negatives)
            cell_color = '#E60000'
        
        # Draw cell as rectangle
        rect = Rectangle((j - 0.5, i - 0.5), 1, 1, 
                         facecolor=cell_color, 
                         edgecolor='black', 
                         linewidth=2)
        ax.add_patch(rect)
        
        # Add text annotations
        if value > 0:
            ax.text(j, i, str(value), ha='center', va='center', 
                   color='white', fontsize=18, fontweight='bold')

# Set axis limits and properties
ax.set_xlim(-0.5, NUM_CLASSES - 0.5)
ax.set_ylim(NUM_CLASSES - 0.5, -0.5)
ax.set_aspect('equal')

# Set ticks and labels
class_labels = [CLASS_NAMES[i] for i in range(NUM_CLASSES)]
ax.set_xticks(np.arange(NUM_CLASSES))
ax.set_yticks(np.arange(NUM_CLASSES))
ax.set_xticklabels(class_labels, fontsize=14, fontweight='bold')
ax.set_yticklabels(class_labels, fontsize=14, fontweight='bold')

# Rotate x labels for better readability
plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')

# Add labels and title
ax.set_xlabel('Predicted Class', fontweight='bold', fontsize=16, labelpad=10)
ax.set_ylabel('True Class', fontweight='bold', fontsize=16, labelpad=10)
ax.set_title(f'Confusion Matrix (From {MODEL_NAME} Validation)\\n(Green = Correct Predictions, Red = Incorrect Predictions, White = No Predictions)', 
             fontweight='bold', fontsize=18, pad=20)

# Remove all gridlines
ax.grid(False)

plt.tight_layout()

# Save confusion matrix figure for PDF report
confusion_matrix_path = RUN_DIR / 'confusion_matrix.png'
plt.savefig(confusion_matrix_path, dpi=150, bbox_inches='tight')
plt.show()

print(f'✓ Confusion matrix visualized (from {MODEL_NAME} validation)')
print(f'  Diagonal sum (correct predictions): {np.trace(confusion_matrix)}')
print(f'  Total predictions: {confusion_matrix.sum()}')
print(f'  Saved to: {confusion_matrix_path}')

## 9. Detailed Comparison: Ground Truth vs Predictions

In [ ]:
def draw_ground_truth(img_path, label_path, class_names, colors):
    """Draw ground truth boxes on image."""
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    if not label_path.exists():
        return img, 0
    
    object_count = 0
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                object_count += 1
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:5])
                
                # Convert to pixel coordinates
                x1 = int((x_center - width / 2) * w)
                y1 = int((y_center - height / 2) * h)
                x2 = int((x_center + width / 2) * w)
                y2 = int((y_center + height / 2) * h)
                
                # Draw box
                color = colors.get(class_id, (255, 255, 255))
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                
                # Draw label
                label = class_names.get(class_id, f'class_{class_id}')
                (label_w, label_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                cv2.rectangle(img, (x1, y1 - label_h - 10), (x1 + label_w + 10, y1), color, -1)
                cv2.putText(img, label, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return img, object_count


# Run predictions on sample images for detailed comparison
print('=' * 80)
print('GENERATING DETAILED GROUND TRUTH VS PREDICTIONS COMPARISON')
print('=' * 80)

# Create directory for comparison images
comparisons_dir = RUN_DIR / 'sample_comparisons'
comparisons_dir.mkdir(exist_ok=True)

# Select 12 random samples
num_comparisons = min(12, len(valid_images))
comparison_indices = random.sample(range(len(valid_images)), num_comparisons)
sample_images = [valid_images[i] for i in comparison_indices]

# Run predictions on sample images
print(f'\nRunning predictions on {num_comparisons} sample images...')
sample_results = []
for img_path in tqdm(sample_images, desc='Processing samples'):
    result = model(str(img_path), verbose=False)[0]
    sample_results.append({
        'image_path': img_path,
        'result': result,
        'num_detections': len(result.boxes)
    })

# Generate 12 separate comparison images (side-by-side)
comparison_image_paths = []
for idx, sample_data in enumerate(sample_results, 1):
    img_path = sample_data['image_path']
    label_path = LABELS_DIR / f'{img_path.stem}.txt'
    result = sample_data['result']
    
    # Ground truth
    gt_img, gt_count = draw_ground_truth(img_path, label_path, CLASS_NAMES, COLORS)
    
    # Predictions
    pred_img = result.plot()
    pred_img_rgb = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)
    
    # Create side-by-side figure for this pair
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # Ground truth
    axes[0].imshow(gt_img)
    axes[0].set_title(
        f'Ground Truth\n{img_path.name}\n({gt_count} objects)', 
        fontweight='bold', fontsize=14, pad=15
    )
    axes[0].axis('off')
    
    # Predictions
    axes[1].imshow(pred_img_rgb)
    axes[1].set_title(
        f'Predictions\n{img_path.name}\n({sample_data["num_detections"]} objects)',
        fontweight='bold', fontsize=14, pad=15
    )
    axes[1].axis('off')
    
    # Add overall title
    fig.suptitle(f'Comparison #{idx}: Ground Truth vs YOLO Predictions', 
                 fontsize=18, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    
    # Save individual comparison
    comparison_path = comparisons_dir / f'comparison_{idx:02d}.png'
    plt.savefig(comparison_path, dpi=150, bbox_inches='tight')
    comparison_image_paths.append(comparison_path)
    
    # Show in notebook
    plt.show()
    
    # Close figure to free memory
    plt.close(fig)

print(f'\n✓ Generated {num_comparisons} individual comparison images')
print(f'  Saved to: {comparisons_dir}')
print(f'  Each image is high-resolution side-by-side comparison')
print('=' * 80)


## 10. Report

## 9.5 Attribute-Based Performance Analysis

Analyze model performance across different environmental conditions and scenes using the metadata.

In [ ]:
if performance_data and image_attributes:
    print("="*80)
    print("ATTRIBUTE-BASED PERFORMANCE ANALYSIS")
    print("="*80)
    
    # Group results by attributes
    results_by_weather = {}
    results_by_scene = {}
    results_by_timeofday = {}
    results_by_class = {}
    
    for result in results_data:
        basename = Path(result['image_path']).stem
        
        # Get attributes for this image
        attrs = image_attributes.get(basename, {})
        weather = attrs.get('weather', 'unknown')
        scene = attrs.get('scene', 'unknown')
        timeofday = attrs.get('timeofday', 'unknown')
        classes_present = attrs.get('classes_present', [])
        
        # Group by weather
        if weather not in results_by_weather:
            results_by_weather[weather] = []
        results_by_weather[weather].append(result)
        
        # Group by scene
        if scene not in results_by_scene:
            results_by_scene[scene] = []
        results_by_scene[scene].append(result)
        
        # Group by time of day
        if timeofday not in results_by_timeofday:
            results_by_timeofday[timeofday] = []
        results_by_timeofday[timeofday].append(result)
        
        # Group by classes present in ground truth
        for cls in classes_present:
            if cls not in results_by_class:
                results_by_class[cls] = []
            results_by_class[cls].append(result)
    
    # Calculate metrics for each group
    def calculate_group_metrics(group_results):
        """Calculate average metrics for a group of results"""
        if not group_results:
            return None
        
        total_tp = sum(r['true_positives'] for r in group_results)
        total_fp = sum(r['false_positives'] for r in group_results)
        total_fn = sum(r['false_negatives'] for r in group_results)
        
        precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
        recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        return {
            'count': len(group_results),
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'tp': total_tp,
            'fp': total_fp,
            'fn': total_fn
        }
    
    # Display weather-based performance
    print("\n" + "="*80)
    print("PERFORMANCE BY WEATHER CONDITION")
    print("="*80)
    weather_metrics = {}
    for weather, group in sorted(results_by_weather.items()):
        metrics = calculate_group_metrics(group)
        if metrics:
            weather_metrics[weather] = metrics
            print(f"\n{weather.upper()}:")
            print(f"  Images: {metrics['count']}")
            print(f"  Precision: {metrics['precision']:.3f}")
            print(f"  Recall: {metrics['recall']:.3f}")
            print(f"  F1 Score: {metrics['f1']:.3f}")
    
    # Display scene-based performance
    print("\n" + "="*80)
    print("PERFORMANCE BY SCENE TYPE")
    print("="*80)
    scene_metrics = {}
    for scene, group in sorted(results_by_scene.items()):
        metrics = calculate_group_metrics(group)
        if metrics:
            scene_metrics[scene] = metrics
            print(f"\n{scene.upper()}:")
            print(f"  Images: {metrics['count']}")
            print(f"  Precision: {metrics['precision']:.3f}")
            print(f"  Recall: {metrics['recall']:.3f}")
            print(f"  F1 Score: {metrics['f1']:.3f}")
    
    # Display time-based performance
    print("\n" + "="*80)
    print("PERFORMANCE BY TIME OF DAY")
    print("="*80)
    timeofday_metrics = {}
    for timeofday, group in sorted(results_by_timeofday.items()):
        metrics = calculate_group_metrics(group)
        if metrics:
            timeofday_metrics[timeofday] = metrics
            print(f"\n{timeofday.upper()}:")
            print(f"  Images: {metrics['count']}")
            print(f"  Precision: {metrics['precision']:.3f}")
            print(f"  Recall: {metrics['recall']:.3f}")
            print(f"  F1 Score: {metrics['f1']:.3f}")
    
    # Display class-based performance
    print("\n" + "="*80)
    print("PERFORMANCE BY CLASS (Top 10 by image count)")
    print("="*80)
    class_metrics = {}
    for cls, group in results_by_class.items():
        metrics = calculate_group_metrics(group)
        if metrics:
            class_metrics[cls] = metrics
    
    # Sort by image count and show top 10
    top_classes = sorted(class_metrics.items(), key=lambda x: x[1]['count'], reverse=True)[:10]
    for cls, metrics in top_classes:
        print(f"\n{cls.upper()}:")
        print(f"  Images: {metrics['count']}")
        print(f"  Precision: {metrics['precision']:.3f}")
        print(f"  Recall: {metrics['recall']:.3f}")
        print(f"  F1 Score: {metrics['f1']:.3f}")
    
    # Visualize attribute-based performance
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Weather performance
    ax = axes[0, 0]
    weather_names = list(weather_metrics.keys())
    weather_f1 = [weather_metrics[w]['f1'] for w in weather_names]
    ax.barh(weather_names, weather_f1, color='skyblue')
    ax.set_xlabel('F1 Score')
    ax.set_title('Performance by Weather Condition')
    ax.set_xlim(0, 1)
    
    # Scene performance
    ax = axes[0, 1]
    scene_names = list(scene_metrics.keys())
    scene_f1 = [scene_metrics[s]['f1'] for s in scene_names]
    ax.barh(scene_names, scene_f1, color='lightcoral')
    ax.set_xlabel('F1 Score')
    ax.set_title('Performance by Scene Type')
    ax.set_xlim(0, 1)
    
    # Time of day performance
    ax = axes[1, 0]
    time_names = list(timeofday_metrics.keys())
    time_f1 = [timeofday_metrics[t]['f1'] for t in time_names]
    ax.barh(time_names, time_f1, color='lightgreen')
    ax.set_xlabel('F1 Score')
    ax.set_title('Performance by Time of Day')
    ax.set_xlim(0, 1)
    
    # Top classes performance
    ax = axes[1, 1]
    class_names_top = [c[0] for c in top_classes[:8]]  # Top 8 for readability
    class_f1_top = [c[1]['f1'] for c in top_classes[:8]]
    ax.barh(class_names_top, class_f1_top, color='plum')
    ax.set_xlabel('F1 Score')
    ax.set_title('Performance by Class (Top 8)')
    ax.set_xlim(0, 1)
    
    plt.tight_layout()
    
    # Save figure
    attribute_perf_path = RUN_DIR / 'attribute_performance.png'
    plt.savefig(attribute_perf_path, dpi=150, bbox_inches='tight')
    print(f"\n✓ Saved attribute performance visualization: {attribute_perf_path}")
    
    plt.show()
    plt.close(fig)
    
    print("\n" + "="*80)
    print("✓ ATTRIBUTE-BASED ANALYSIS COMPLETE")
    print("="*80)
else:
    print("="*80)
    print("⚠️  Attribute-based analysis not available")
    print("Performance metadata not loaded")
    print("="*80)

In [ ]:
from datetime import datetime
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from PIL import Image as PILImage

# Generate comprehensive PDF report with all visualizations
pdf_report_path = RUN_DIR / 'report.pdf'

# Create PDF document
doc = SimpleDocTemplate(str(pdf_report_path), pagesize=A4,
                       rightMargin=30, leftMargin=30,
                       topMargin=30, bottomMargin=30)

# Container for PDF elements
story = []
styles = getSampleStyleSheet()

# Custom styles
title_style = ParagraphStyle(
    'CustomTitle',
    parent=styles['Heading1'],
    fontSize=24,
    textColor=colors.HexColor('#2c3e50'),
    spaceAfter=30,
    alignment=TA_CENTER
)

heading_style = ParagraphStyle(
    'CustomHeading',
    parent=styles['Heading2'],
    fontSize=16,
    textColor=colors.HexColor('#34495e'),
    spaceAfter=12,
    spaceBefore=20
)

# Title
story.append(Paragraph('YOLO Validation Report', title_style))
story.append(Spacer(1, 12))

# Info section with model details
info_data = [
    ['Model:', MODEL_NAME],
    ['Model Size:', f'{model_size_mb:.1f} MB'],
    ['Parameters:', f'{model_params / 1e6:.1f}M'],
    ['FLOPs (640x640):', f'{flops_gflops:.2f} GFLOPs'],
    ['Run Name:', RUN_NAME],
    ['W&B Run Name:', W_B_RUN_NAME],
    ['Timestamp:', datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
    ['Dataset:', f'{USED_DATASET} - {USED_DATA_SPLIT} split'],
    ['Images Processed:', str(len(results_data))],
    ['IoU Threshold:', str(iou_threshold)]
]

info_table = Table(info_data, colWidths=[2.2*inch, 3.8*inch])
info_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, -1), colors.HexColor('#ecf0f1')),
    ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ('TOPPADDING', (0, 0), (-1, -1), 8),
    ('GRID', (0, 0), (-1, -1), 1, colors.white)
]))
story.append(info_table)
story.append(Spacer(1, 20))

# Performance metrics section
story.append(Paragraph('Inference Performance', heading_style))
perf_data = [
    ['Metric', 'Value'],
    ['Total Execution Time', f'{total_time:.2f}s'],
    ['Average Inference Time', f'{avg_inference_time*1000:.2f}ms per image'],
    ['FPS (Frames Per Second)', f'{fps:.2f}']
]

perf_table = Table(perf_data, colWidths=[3*inch, 3*inch])
perf_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#27ae60')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 12),
    ('FONTSIZE', (0, 1), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ('TOPPADDING', (0, 0), (-1, -1), 8),
    ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#d5f4e6')),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
]))
story.append(perf_table)
story.append(Spacer(1, 20))

# Overall accuracy metrics
story.append(Paragraph('Overall Accuracy Metrics', heading_style))
acc_data = [
    ['Metric', 'Value'],
    ['Precision', f'{overall_precision:.4f}'],
    ['Recall', f'{overall_recall:.4f}'],
    ['F1-Score', f'{overall_f1:.4f}'],
    ['mAP@0.5', f'{yolo_metrics["map50"]:.4f}'],
    ['mAP@0.5:0.95', f'{yolo_metrics["map50_95"]:.4f}'],
    ['True Positives', str(total_tp)],
    ['False Positives', str(total_fp)],
    ['False Negatives', str(total_fn)]
]

acc_table = Table(acc_data, colWidths=[3*inch, 3*inch])
acc_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#3498db')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 12),
    ('FONTSIZE', (0, 1), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ('TOPPADDING', (0, 0), (-1, -1), 8),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
]))
story.append(acc_table)
story.append(Spacer(1, 20))

# Add metrics visualization to PDF
story.append(PageBreak())
story.append(Paragraph('Performance Visualizations', heading_style))
story.append(Spacer(1, 12))

metrics_fig_path = RUN_DIR / 'metrics_charts.png'
if metrics_fig_path.exists():
    try:
        with PILImage.open(metrics_fig_path) as img:
            img_width, img_height = img.size
            aspect_ratio = img_height / img_width
            pdf_width = 7 * inch
            pdf_height = pdf_width * aspect_ratio
            story.append(Image(str(metrics_fig_path), width=pdf_width, height=pdf_height))
    except Exception as e:
        print(f'Warning: Could not load metrics chart with PIL: {e}')
        story.append(Image(str(metrics_fig_path), width=7*inch, height=5*inch))
else:
    story.append(Paragraph('Metrics chart not found.', styles['Normal']))

story.append(Spacer(1, 20))

# Per-class metrics table
story.append(PageBreak())
story.append(Paragraph('Per-Class Performance', heading_style))

table_data = [['Class', 'TP', 'FP', 'FN', 'Precision', 'Recall', 'F1-Score', 'mAP@0.5']]
for _, row in df_metrics.iterrows():
    class_name = row['Class']
    map50_val = yolo_class_metrics.get(class_name, {}).get('ap50', 0.0)
    table_data.append([
        row['Class'],
        str(row['TP']),
        str(row['FP']),
        str(row['FN']),
        f"{row['Precision']:.4f}",
        f"{row['Recall']:.4f}",
        f"{row['F1-Score']:.4f}",
        f"{map50_val:.4f}"
    ])

per_class_table = Table(table_data, colWidths=[1.0*inch, 0.5*inch, 0.5*inch, 0.5*inch, 0.8*inch, 0.8*inch, 0.8*inch, 0.8*inch])
per_class_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#3498db')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 8),
    ('FONTSIZE', (0, 1), (-1, -1), 7),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('TOPPADDING', (0, 0), (-1, -1), 6),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
]))
story.append(per_class_table)
story.append(Spacer(1, 20))

# Confusion Matrix
story.append(PageBreak())
story.append(Paragraph('Confusion Matrix', heading_style))
story.append(Paragraph(f'Correct Predictions (Diagonal Sum): {np.trace(confusion_matrix)}', styles['Normal']))
story.append(Paragraph(f'Total Matched Predictions: {confusion_matrix.sum()}', styles['Normal']))
story.append(Spacer(1, 12))

confusion_matrix_img_path = RUN_DIR / 'confusion_matrix.png'
if confusion_matrix_img_path.exists():
    try:
        with PILImage.open(confusion_matrix_img_path) as img:
            img_width, img_height = img.size
            aspect_ratio = img_height / img_width
            pdf_width = 6.5 * inch
            pdf_height = pdf_width * aspect_ratio
            if pdf_height > 7 * inch:
                pdf_height = 7 * inch
                pdf_width = pdf_height / aspect_ratio
            story.append(Image(str(confusion_matrix_img_path), width=pdf_width, height=pdf_height))
    except Exception as e:
        print(f'Warning: Could not load confusion matrix with PIL: {e}')
        story.append(Image(str(confusion_matrix_img_path), width=6.5*inch, height=6.5*inch))
else:
    story.append(Paragraph('Confusion matrix image not found.', styles['Normal']))

story.append(Spacer(1, 20))

# Sample predictions comparison - multiple images per page
comparisons_dir = RUN_DIR / 'sample_comparisons'
if comparisons_dir.exists():
    comparison_files = sorted(comparisons_dir.glob('comparison_*.png'))
    
    if comparison_files:
        story.append(PageBreak())
        story.append(Paragraph('Sample Predictions: Ground Truth vs Predictions', heading_style))
        story.append(Paragraph(f'{len(comparison_files)} detailed side-by-side comparisons', styles['Normal']))
        story.append(Spacer(1, 12))
        
        # Add comparison images - 2 per page with page breaks between pairs
        for i, comparison_path in enumerate(comparison_files):
            # Add page break after every 3 images (except before the first)
            if i > 0 and i % 3 == 0:
                story.append(PageBreak())
            
            try:
                with PILImage.open(comparison_path) as img:
                    img_width, img_height = img.size
                    aspect_ratio = img_height / img_width
                    
                    # Smaller width to fit 2 per page
                    pdf_width = 7 * inch
                    pdf_height = pdf_width * aspect_ratio
                    
                    # Limit height to fit 2 images per page
                    if pdf_height > 3.5 * inch:
                        pdf_height = 3.5 * inch
                        pdf_width = pdf_height / aspect_ratio
                    
                    story.append(Image(str(comparison_path), width=pdf_width, height=pdf_height))
                    story.append(Spacer(1, 8))
            except Exception as e:
                print(f'Warning: Could not load {comparison_path.name} with PIL: {e}')
                story.append(Image(str(comparison_path), width=7*inch, height=3*inch))
                story.append(Spacer(1, 8))
    else:
        story.append(PageBreak())
        story.append(Paragraph('Sample Predictions: Ground Truth vs Predictions', heading_style))
        story.append(Paragraph('No comparison images found.', styles['Normal']))
else:
    story.append(PageBreak())
    story.append(Paragraph('Sample Predictions: Ground Truth vs Predictions', heading_style))
    story.append(Paragraph('Comparison directory not found.', styles['Normal']))

story.append(Spacer(1, 12))
story.append(Paragraph('Additional validation plots available in: yolo_validation folder', styles['Normal']))

# Footer
story.append(Spacer(1, 30))
story.append(Paragraph('Generated by YOLO Quick Test Notebook', 
                      ParagraphStyle('Footer', parent=styles['Normal'], 
                                   alignment=TA_CENTER, textColor=colors.grey)))
story.append(Paragraph('BDD100K Dataset - Computer Vision Project', 
                      ParagraphStyle('Footer2', parent=styles['Normal'], 
                                   alignment=TA_CENTER, textColor=colors.grey)))

# Build PDF
doc.build(story)

# Generate JSON file with comprehensive comparison data
json_report_path = RUN_DIR / 'metrics_data.json'

comparison_data = {
    'metadata': {
        'model_name': MODEL_NAME,
        'run_name': RUN_NAME,
        'wb_run_name': W_B_RUN_NAME,
        'timestamp': datetime.now().isoformat(),
        'dataset': USED_DATASET,
        'data_split': USED_DATA_SPLIT,
        'images_processed': len(results_data),
        'iou_threshold': iou_threshold,
        'num_classes': NUM_CLASSES
    },
    'model_info': {
        'parameters': int(model_params),
        'model_size_mb': float(model_size_mb),
        'flops_gflops': float(flops_gflops)
    },
    'performance': {
        'total_time_seconds': float(total_time),
        'avg_inference_time_ms': float(avg_inference_time * 1000),
        'fps': float(fps),
        'images_processed': int(len(results_data))
    },
    'custom_metrics': {
        'overall': {
            'precision': float(overall_precision),
            'recall': float(overall_recall),
            'f1_score': float(overall_f1),
            'true_positives': int(total_tp),
            'false_positives': int(total_fp),
            'false_negatives': int(total_fn)
        },
        'per_class': {}
    },
    'yolo_official_metrics': {
        'overall': yolo_metrics,
        'per_class': yolo_class_metrics
    },
    'confusion_matrix': {
        'matrix': confusion_matrix.tolist(),
        'diagonal_sum': int(np.trace(confusion_matrix)),
        'total_predictions': int(confusion_matrix.sum())
    },
    'class_names': CLASS_NAMES
}

# Add per-class custom metrics
for _, row in df_metrics.iterrows():
    class_name = row['Class']
    comparison_data['custom_metrics']['per_class'][class_name] = {
        'true_positives': int(row['TP']),
        'false_positives': int(row['FP']),
        'false_negatives': int(row['FN']),
        'precision': float(row['Precision']),
        'recall': float(row['Recall']),
        'f1_score': float(row['F1-Score'])
    }

# Save JSON file
import json
with open(json_report_path, 'w') as f:
    json.dump(comparison_data, f, indent=2)

# Count comparison images
comparisons_dir = RUN_DIR / 'sample_comparisons'
num_comparison_images = len(list(comparisons_dir.glob('comparison_*.png'))) if comparisons_dir.exists() else 0

# Summary output
print('=' * 80)
print('✓ COMPREHENSIVE REPORT GENERATED')
print('=' * 80)
print(f'\nAll outputs saved to: {RUN_DIR}')
print(f'\nGenerated files:')
print(f'  📄 PDF Report: report.pdf ({pdf_report_path.stat().st_size / 1024:.2f} KB)')
print(f'  📊 Metrics JSON: metrics_data.json ({json_report_path.stat().st_size / 1024:.2f} KB)')
print(f'  📈 YOLO Validation: yolo_validation_metrics.json')
print(f'  🖼️  Confusion Matrix: confusion_matrix.png')
print(f'  📊 Metrics Charts: metrics_charts.png')
print(f'  🖼️  Sample Comparisons: sample_comparisons/ ({num_comparison_images} images)')
print(f'  📁 YOLO Outputs: yolo_validation/ (plots, curves, etc.)')
print(f'\nReport contents:')
print(f'  - Model Information (Size: {model_size_mb:.1f}MB, Params: {model_params/1e6:.1f}M, FLOPs: {flops_gflops:.2f}G)')
print(f'  - Performance Metrics (FPS: {fps:.2f}, Avg: {avg_inference_time*1000:.2f}ms)')
print(f'  - Overall accuracy metrics (Precision, Recall, F1, mAP)')
print(f'  - Per-class performance with IoU metrics')
print(f'  - Performance visualizations (4 charts)')
print(f'  - Confusion matrix visualization')
print(f'  - {num_comparison_images} individual side-by-side prediction comparisons (full-width, high-resolution)')
print(f'  - Custom & YOLO official metrics')
print(f'\n💾 Metrics saved to JSON for future comparison')

print(f'📊 Weights & Biases tracking: {W_B_RUN_NAME}')
print('=' * 80)